In [65]:
import pandas as pd
import GridPoint as gd
import xml.etree.ElementTree as ET
import configparser


In [73]:
df = pd.read_csv("raw data files/Grid-Data.csv", header=None)

In [74]:
df.head()

,0,1
0,0.0 0.0,eduroam a0:b4:39:54:8c:6e -90 UCDWireless a0:b...
1,1.0 0.0,eduroam a0:b4:39:54:8c:6e -92 UCDWireless a0:b...
2,2.0 0.0,eduroam a0:b4:39:54:8c:6e -91 UCDWireless a0:b...
3,3.0 0.0,eduroam a0:b4:39:54:8c:6e -92 UCDWireless a0:b...
4,4.0 0.0,eduroam a0:b4:39:54:8c:6e -92 UCDWireless a0:b...


In [77]:
class GridPoint:
    def __init__(self, x, y, data):
        self.x = x
        self.y = y
        self.data = data
        
    def __str__(self):
        return f"{self.x}, {self.y}"

gridPoints = []

for i in range(40):
    x = df[0][i].split()[0]
    y = df[0][i].split()[1]
    data = df[1][i].split()
    mlist = []
    for j in range(len(data)):
        if j % 3 == 0:
            thisdict = dict(address = data[j+1], rssi = data[j+2], ssid = data[j])
            mlist.append(thisdict)
    
    gridPoints.append(GridPoint(x, y, mlist))
    #list.append(GridPoint(x, y, mlist))

In [78]:
gridPoints[0].data

[{'address': 'a0:b4:39:54:8c:6e', 'rssi': '-90', 'ssid': 'eduroam'},
 {'address': 'a0:b4:39:54:8c:6d', 'rssi': '-90', 'ssid': 'UCDWireless'},
 {'address': 'a0:b4:39:44:e8:81', 'rssi': '-79', 'ssid': 'eduroam'},
 {'address': 'a0:b4:39:37:2d:0e', 'rssi': '-81', 'ssid': 'eduroam'},
 {'address': 'a0:b4:39:37:2d:0d', 'rssi': '-81', 'ssid': 'UCDWireless'},
 {'address': 'a0:b4:39:28:87:8e', 'rssi': '-80', 'ssid': 'eduroam'},
 {'address': 'a0:b4:39:28:87:8d', 'rssi': '-80', 'ssid': 'UCDWireless'},
 {'address': 'a0:b4:39:33:3e:2e', 'rssi': '-90', 'ssid': 'eduroam'},
 {'address': 'a0:b4:39:37:2f:81', 'rssi': '-79', 'ssid': 'eduroam'},
 {'address': '78:f1:c6:3e:41:8e', 'rssi': '-91', 'ssid': 'eduroam'},
 {'address': '78:f1:c6:3e:41:8d', 'rssi': '-91', 'ssid': 'UCDWireless'},
 {'address': 'a0:b4:39:28:1f:6d', 'rssi': '-86', 'ssid': 'UCDWireless'},
 {'address': 'a0:b4:39:28:1f:6e', 'rssi': '-86', 'ssid': 'eduroam'},
 {'address': '3c:51:0e:ac:e9:0e', 'rssi': '-87', 'ssid': 'eduroam'},
 {'address': '

In [ ]:
df1 = grid_points[0].df
df2 = grid_points[1].df


test1 = df1.set_index("mac_ad").to_dict()["rssi"]
test2 = df2.set_index("mac_ad").to_dict()["rssi"]
print(test1)
print(test2)

In [34]:
dict_1 = df1.set_index("mac_ad").to_dict()["rssi"]
dict_2 = df2.set_index("mac_ad").to_dict()["rssi"]
dict_3 = {}
for key, value in dict_2.items():
    if key in dict_1:
        if isinstance(dict_1[key], list):
            dict_1[key].append(value)
        else:
            temp_list = [dict_1[key]]
            temp_list.append(value)
            dict_1[key] = temp_list
    else:
        dict_1[key] = value

dict_1

NameError: name 'df1' is not defined

In [35]:
rejects = {}
def f(k, v):
    if isinstance(v, list):
        return abs(int(v[0])-int(v[1]))
    else:
        rejects[k] = v

dict = {k: f(k, v) for k, v in dict_1.items()}
[dict.pop(key) for key in rejects.keys()]
error = sum(dict.values())
error

NameError: name 'dict_1' is not defined

In [36]:
import configparser
import GridPoint as gd
import pandas as pd
import DataModel


#read config file
config = configparser.ConfigParser()
config.read("config.ini")
config_paths = config["paths"]
config_grid = config["grid"]

#config params
numGridPoints = int(config_grid["numGridPoints"])

grid_points = DataModel.DataModel("test.xml", 10)#Contains XML data from training map
grid_points_test = DataModel.DataModel("testy.xml", 5)#Contains XML data from random locations (Currently 5)

#Sum of Errors list
all_errors = []

#Match every position  with every grid point.
#MAC Addresses that do not match are currently not used as extra data
for test_point in grid_points_test:
    errors = []
    for grid_point in grid_points:
        df_test = test_point.df
        df_data = grid_point.df
        
        #Dictionaries to extract unique key-value pairs from mac address and rssi from both lists
        #dict_1 will store all key-value pairs that do and do not match
        dict_1 = df_data.set_index("mac_ad").to_dict()["rssi"]
        dict_2 = df_test.set_index("mac_ad").to_dict()["rssi"]
        for key, value in dict_2.items():
            if key in dict_1:
                if isinstance(dict_1[key], list):
                    dict_1[key].append(value)
                else:
                    temp_list = [dict_1[key]]
                    temp_list.append(value)
                    dict_1[key] = temp_list
            else:
                dict_1[key] = value

                
        #This list is all key value pairs that did not match in both lists. Will be used for later
        rejects = {}
        #conglomerates
        def f(k, v):
            if isinstance(v, list):
                return abs(int(v[0]) - int(v[1]))
            else:
                rejects[k] = v

                
        dict = {k: f(k, v) for k, v in dict_1.items()}
        [dict.pop(key) for key in rejects.keys()]

        #
        error = sum(dict.values())
        errors.append(error)

    all_errors.append(errors)
print(all_errors)

[[94, 98, 114, 104, 105, 88, 68, 83, 81, 75], [78, 45, 68, 90, 67, 55, 39, 53, 71, 43], [72, 84, 69, 75, 61, 53, 81, 109, 111, 91], [19, 47, 48, 41, 46, 47, 58, 94, 76, 70], [45, 60, 68, 44, 56, 63, 77, 88, 72, 59]]


In [52]:
tree = ET.parse("test.xml")
root = tree.getroot()


one = all_errors[0]
sorted_one = sorted(range(len(one)), key=lambda k: one[k])

print(sorted_one)
coord_list = []
for i in range(1):
    grid = root[sorted_one[i]]
    x = int(grid.find("X").text)
    y = int(grid.find("Y").text)
    coord_list.append([x,y])
    
print(coord_list)

x = 0
y = 0
for coord in coord_list:
    x += coord[0]
    y += coord[1]
x /= len(coord_list)
y /= len(coord_list)
print(f"Predicted Coordinates: {x},{y}")

[6, 9, 8, 7, 5, 0, 1, 3, 4, 2]
[[4, 26]]
Predicted Coordinates: 4.0,26.0


In [38]:
import xml.etree.ElementTree as ET
import configparser

file = "xml\data.xml"
grid_id=0
#read config file
config = configparser.ConfigParser()
config.read("config.ini")
config_grid_points = config["grid_points"]

# parse tree from xml file
tree = ET.parse(file)
root = tree.getroot()


grid = root[grid_id]
x = root[grid_id].find("X")
x.text = eval(config_grid_points[str(grid_id)])["y"]
print(eval(config_grid_points[str(grid_id)])["y"])
y = root[grid_id].find("Y")
y.text = eval(config_grid_points[str(grid_id)])["y"]

18
